In [1]:
import numpy as np
from helper_methods import *

import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F


In [2]:
A_s = np.array([[0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0.5, 0.5, 0.5, 0, 0.5, 0],
                [0, 1, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0]], dtype=np.float64)

print(A_s.shape)

(4, 8)


In [3]:

W_z = get_initial_weight(A_s)
W_r = get_initial_weight(A_s)
W_o = np.random.rand()
U_z = get_initial_weight(A_s)
U_r = get_initial_weight(A_s)
U_o = np.random.rand()

In [4]:
H = get_initial_weight(A_s)

print(H)

[[0.79014953 0.20140947 0.30774749 0.9367496  0.8634292  0.80524586
  0.46889953 0.69859226]
 [0.04495939 0.07704207 0.34636242 0.13558391 0.44875879 0.35430877
  0.68643949 0.58172166]
 [0.21323418 0.63166635 0.78133131 0.97171442 0.91557864 0.01890567
  0.37423972 0.13062085]
 [0.09340643 0.17404221 0.95878914 0.0029192  0.11298211 0.47056422
  0.51584241 0.39677447]]


In [5]:
b = np.ones(A_s.shape)

print(b)

[[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]


$a_{s, i}^t=A_{s,i:}\left [ v_{1}^{t-1}, \dots , v_n^{t-1} \right ]$

In [6]:
a_s = np.zeros(A_s.shape)

for i in range(len(A_s)):
    a_s[i] = np.matmul(np.transpose(A_s[i]), H[i]) + b[i]
    

print(a_s.shape)

(4, 8)


In [7]:
z_s = np.ones(A_s.shape)


for i in range(len(a_s)):
    z_s[i] = activation_function_on_vector(sigmoid, (W_z[i] * a_s[i]) + (U_z[i] * A_s[i]))

print(z_s)

[[0.52006431 0.74472711 0.68086588 0.75422975 0.54108794 0.57430019
  0.53971416 0.61040884]
 [0.62916084 0.75469954 0.82293308 0.8211417  0.85374494 0.75700586
  0.82614799 0.81573307]
 [0.53238098 0.83152569 0.58756334 0.5770706  0.73385152 0.87588891
  0.50828674 0.78375006]
 [0.58592828 0.80311553 0.77106956 0.6643751  0.6805503  0.78776634
  0.69235297 0.5895793 ]]


In [8]:
r_s = np.ones(A_s.shape)

for i in range(len(a_s)):
    r_s[i] = activation_function_on_vector(sigmoid, (W_r[i] * a_s[i]) + (U_r[i] * A_s[i]))

print(r_s)

[[0.76147839 0.61838129 0.64643254 0.52588077 0.5642069  0.63386067
  0.71065664 0.60043094]
 [0.8439171  0.74108967 0.83702022 0.70762391 0.84685634 0.84316218
  0.74092596 0.58238254]
 [0.72303528 0.77589315 0.69057898 0.5833038  0.74349568 0.85823104
  0.73952856 0.51326559]
 [0.72860648 0.51784542 0.81072467 0.59429167 0.74277466 0.66442447
  0.51778211 0.78350547]]


In [43]:
hidden_size = 8
input_size = hidden_size * 2
gate_size = 3 * hidden_size
n_nodes = 4

In [45]:
embedding = nn.Embedding(n_nodes, hidden_size)

In [57]:
v_hidden = np.ones(A_s.shape)



w_ih = Parameter(torch.Tensor(gate_size, input_size))
w_hh = Parameter(torch.Tensor(gate_size, hidden_size))
b_ih = Parameter(torch.Tensor(gate_size))
b_hh = Parameter(torch.Tensor(gate_size))
b_iah = Parameter(torch.Tensor(hidden_size))
b_oah = Parameter(torch.Tensor(hidden_size))

print(w_ih.shape)
print(w_hh.shape)
print(b_ih.shape)
print(b_hh.shape)
print(b_iah.shape)
print(b_oah.shape)

linear_edge_in = nn.Linear(hidden_size, hidden_size, bias=True)
linear_edge_out = nn.Linear(hidden_size, hidden_size, bias=True)
linear_edge_f = nn.Linear(hidden_size, hidden_size, bias=True)

print(linear_edge_in)

A_s = torch.tensor([[0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0.5, 0.5, 0.5, 0, 0.5, 0],
                [0, 1, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0]], dtype=torch.float32)



#hidden = torch.tensor(np.array(np.transpose(H)))
#hidden = torch.tensor(np.array(H))
hidden = torch.randn(4, 8)
nn.Embedding
tensor_A_s = torch.transpose(A_s, 0, 1)
#tensor_A_s = torch.randn(8,4)
print(tensor_A_s.dtype)
print(hidden.dtype)

input_in = torch.matmul(tensor_A_s[:, :A_s.shape[1]], linear_edge_in(hidden)) + b_iah
input_out = torch.matmul(tensor_A_s[tensor_A_s.shape[1]: 2 * tensor_A_s.shape[1]], linear_edge_out(hidden)) + b_oah

print(input_in,input_in.shape)
#for i in range(len(A_s)):
#    v_hidden[i] = activation_function_on_vector(tahn, (W_o * ))
tensor_A_s[:, tensor_A_s.shape[1]: 2 * tensor_A_s.shape[1]]

gi = F.linear(inputs, w_ih, b_ih)
gh = F.linear(hidden, w_hh, b_hh)

torch.Size([24, 16])
torch.Size([24, 8])
torch.Size([24])
torch.Size([24])
torch.Size([8])
torch.Size([8])
Linear(in_features=8, out_features=8, bias=True)
torch.float32
torch.float32
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -2.1632e-01,  1.3370e+22,  6.6453e-07],
        [ 9.6689e-01, -3.0732e-01,  1.6482e-01,  1.1635e+00,  7.5914e-01,
         -6.5589e-01,  1.3370e+22,  8.5932e-01],
        [-6.8738e-02,  1.7811e-01, -4.3482e-01,  1.1654e-01, -6.9302e-04,
         -1.0054e-01,  1.3370e+22, -2.1253e-01],
        [-6.8738e-02,  1.7811e-01, -4.3482e-01,  1.1654e-01, -6.9302e-04,
         -1.0054e-01,  1.3370e+22, -2.1253e-01],
        [-6.8738e-02,  1.7811e-01, -4.3482e-01,  1.1654e-01, -6.9302e-04,
         -1.0054e-01,  1.3370e+22, -2.1253e-01],
        [ 4.3075e-01,  2.3825e-01, -1.4423e-02,  4.3806e-01,  3.8228e-01,
         -2.9145e-01,  1.3370e+22, -2.6368e-01],
        [-6.8738e-02,  1.7811e-01, -4.3482e-01,  1.1654e-01, -6.9302e-04,
    

NameError: name 'inputs' is not defined